## Highest Cost Orders `Shopify`

Find the customers with the highest daily total order cost between 2019-02-01 and 2019-05-01. If a customer had more than one order on a certain day, sum the order costs on a daily basis. Output each customer's first name, total cost of their items, and the date.


For simplicity, you can assume that every first name in the dataset is unique.

In [ ]:
# Import your libraries
import pandas as pd

orders[orders['order_date'].between('2019-02-01','2019-05-01')].merge(customers,left_on='cust_id',right_on='id').groupby(['first_name','order_date'])['total_order_cost'].sum().reset_index().sort_values(by='total_order_cost',ascending=False).head(2)

## New Products `Tesla`

Calculate the net change in the number of products launched by companies in 2020 compared to 2019. Your output should include the company names and the net difference.
(Net difference = Number of products launched in 2020 - The number launched in 2019.)

In [ ]:
# Import your libraries
import pandas as pd

# car_launches.groupby(['year','company_name'],as_index=False)['product_name'].count()
car2019 = car_launches.query('year == 2019').groupby('company_name')['product_name'].nunique()
car2020 = car_launches.query('year == 2020').groupby('company_name')['product_name'].nunique()

(car2020 - car2019).reset_index()

## Users By Average Session Time `Meta`

Calculate each user's average session time, where a session is defined as the time difference between a page_load and a page_exit. Assume each user has only one session per day. If there are multiple page_load or page_exit events on the same day, use only the latest page_load and the earliest page_exit, ensuring the page_load occurs before the page_exit. Output the user_id and their average session time.

In [ ]:
# Import your libraries
import pandas as pd

facebook_web_log['day']  = facebook_web_log['timestamp'].dt.day

df = facebook_web_log[~facebook_web_log['action'].str.contains('scroll')]

df = df.groupby(['user_id','day','action'],as_index=False)['timestamp'].max()

df = df.pivot_table(index=['user_id','day'],columns='action',values='timestamp').reset_index().dropna()

df['duration'] = df['page_exit'] - df['page_load']
df.groupby('user_id',as_index=False)['duration'].mean()

## Acceptance Rate By Date `Meta`

Calculate the friend acceptance rate for each date when friend requests were sent. A request is sent if action = sent and accepted if action = accepted. If a request is not accepted, there is no record of it being accepted in the table. The output will only include dates where requests were sent and at least one of them was accepted, as the acceptance rate can only be calculated for those dates. Show the results ordered from the earliest to the latest date.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = fb_friend_requests.pivot_table(index=['user_id_sender','user_id_receiver'],columns='action',values='date').reset_index()

sent = pd.DataFrame(columns=['date','sent'])

sent[['date','sent']] = df['sent'].value_counts().reset_index()

accepted = df.groupby('sent',as_index=False)['accepted'].count()

final = accepted.merge(sent,left_on='sent',right_on='date')

final['acceptance_rate'] = final['accepted']/final['sent_y']

final[['date','acceptance_rate']]

## Finding User Purchases `Amazon`

Identify returning active users by finding users who made a second purchase within 1 to 7 days after their first purchase. Ignore same-day purchases. Output a list of these user_ids.


In [ ]:
# Import your libraries
import pandas as pd

# amazon_transactions[['shifted_users','shifted_created']] =
df = amazon_transactions.sort_values(by=['user_id','created_at'])
df[['shifted_users','shifted_created']] = df[['user_id','created_at']].shift(1)
df = df[(df['user_id'] == df['shifted_users']) | (df['shifted_users'].isna())]

df['delta'] = ((df['created_at'] - df['shifted_created'])/3600)/24

df['delta'] = df['delta'].dt.seconds

df = df.query('delta != 0')

df = df.dropna().drop_duplicates(subset='user_id')
df[df['delta'] < 8]['user_id']

## Premium vs Freemium `Microsoft`

Find the total number of downloads for paying and non-paying users by date. Include only records where non-paying customers have more downloads than paying customers. The output should be sorted by earliest date first and contain 3 columns date, non-paying downloads, paying downloads. Hint: In Oracle you should use "date" when referring to date column (reserved keyword).

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = ms_user_dimension.merge(ms_acc_dimension,on='acc_id').merge(ms_download_facts,on='user_id').groupby(['date','paying_customer'],as_index=False)['downloads'].sum()

df = df.pivot_table(index='date',columns='paying_customer',values='downloads').reset_index()

df.query('no > yes')

## Risky Projects `Linkedin`

You are given a set of projects and employee data. Each project has a name, a budget, and a specific duration, while each employee has an annual salary and may be assigned to one or more projects for particular periods. The task is to identify which projects are overbudget. A project is considered overbudget if the prorated cost of all employees assigned to it exceeds the project’s budget.


To solve this, you must prorate each employee's annual salary based on the exact period they work on a given project, relative to a full year. For example, if an employee works on a six-month project, only half of their annual salary should be attributed to that project. Sum these prorated salary amounts for all employees assigned to a project and compare the total with the project’s budget.


Your output should be a list of overbudget projects, where each entry includes the project’s name, its budget, and the total prorated employee expenses for that project. The total expenses should be rounded up to the nearest dollar. Assume all years have 365 days and disregard leap years.

In [ ]:
# Import your libraries
import pandas as pd
import numpy as np
from math import ceil

# Start writing coded
linkedin_projects['duration'] = ((linkedin_projects['end_date'] - linkedin_projects['start_date']).dt.days)

df = linkedin_projects.merge(linkedin_emp_projects,left_on='id',right_on='project_id').drop(columns='id').merge(linkedin_employees,left_on='emp_id',right_on='id')

df['prorated'] = (df['salary']/365)*(df['duration'])

df = df.groupby('title',as_index=False).agg(
    {
        'budget' : 'max',
        'prorated' : 'sum'
    })

df = df.query('prorated > budget')

df['prorated'] = df['prorated'].apply(lambda x : ceil(x))

df

## Activity Rank `Google`

Find the email activity rank for each user. Email activity rank is defined by the total number of emails sent. The user with the highest number of emails sent will have a rank of 1, and so on. Output the user, total emails, and their activity rank.


•	Order records first by the total emails in descending order.
•	Then, sort users with the same number of emails in alphabetical order by their username.
•	In your rankings, return a unique value (i.e., a unique rank) even if multiple users have the same number of emails.


In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = google_gmail_emails.groupby('from_user',as_index=False)['to_user'].count()

df = df.sort_values(by=['to_user','from_user'],ascending=[False,True])

df['rank'] = df['to_user'].rank(ascending=False,method='first')

df

## Processed Ticket Rate By Type `Meta`

Find the processed rate of tickets for each type. The processed rate is defined as the number of processed tickets divided by the total number of tickets for that type. Round this result to two decimal places.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = facebook_complaints.groupby('type',as_index=False).agg(
    {'processed' : 'sum',
    'complaint_id' : 'count'
    }
    )

df['processed_rate'] = df['processed']/df['complaint_id']

df[['type','processed_rate']]

## Customer Revenue In March `Amazon`

Calculate the total revenue from each customer in March 2019. Include only customers who were active in March 2019. An active user is a customer who made at least one transaction in March 2019.


Output the revenue along with the customer id and sort the results based on the revenue in descending order.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
(
    orders
    .query('order_date >= "2019-03" & order_date < "2019-04"')
    .groupby('cust_id',as_index=False)['total_order_cost'].sum()
    .sort_values(by='total_order_cost',ascending=False)
)

## Count Occurrences Of Words In Drafts `Google`

Find the number of times each word appears in the contents column across all rows in the google_file_store dataset. Output two columns: word and occurrences.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
google_file_store['contents'] = google_file_store['contents'].str.split(' ')
dic = {}
for i in google_file_store['contents'].sum():
    i = i.replace('.','').replace(',','').lower()
    if i in dic:
        dic[i] += 1
    else:
        dic[i] = 1

df = pd.DataFrame(columns=['word','occurences'])

df['word'] = dic.keys()

df['occurences'] = dic.values()
df.sort_values(by='occurences',ascending=False)

## Titanic Survivors and Non-Survivors `Tesla`

Make a report showing the number of survivors and non-survivors by passenger class. Classes are categorized based on the pclass value as:


•	First class: pclass = 1
•	Second class: pclass = 2
•	Third class: pclass = 3


Output the number of survivors and non-survivors by each class.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code

titanic.pivot_table(index='survived',columns='pclass',values='ticket',aggfunc='count').reset_index()

## Second Highest Salary `Dropbox`

Find the second highest salary of employees.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
employee['salary'].sort_values(ascending=False).iloc[1]

## Employee and Manager Salaries `Walmart`


Find employees who are earning more than their managers. Output the employee's first name along with the corresponding salary.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
employee.merge(employee,left_on='manager_id',right_on='id').query('salary_x > salary_y')[['first_name_x','salary_x']]

## Highest Salary In Department `Asana`

Find the employee with the highest salary per department.
Output the department name, employee's first name along with the corresponding salary.

In [ ]:
# Import your libraries
import pandas as pd

data = []
# Start writing code
for i,j in employee.groupby('department'):
    salary = j['salary'].max()
    name = j.query(f'salary == {salary}')['first_name'].values[0]
    data.append({'department' : i,'name' : name,'salary':salary})

pd.DataFrame(data)

## Highest Target Under Manager `Salesforce`

Identify the employee(s) working under manager manager_id=13 who have achieved the highest target. Return each such employee’s first name alongside the target value. The goal is to display the maximum target among all employees under manager_id=13 and show which employee(s) reached that top value.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = salesforce_employees.query('manager_id == 13')

df[df['target'] == df['target'].max()][['first_name','target']]

## Largest Olympics `ESPN`

Find the Olympics with the highest number of unique athletes. The Olympics game is a combination of the year and the season, and is found in the games column. Output the Olympics along with the corresponding number of athletes. The id column uniquely identifies an athlete.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
olympics_athletes_events['oly'] = olympics_athletes_events['year'].astype(str) + ' ' + olympics_athletes_events['season'].astype(str)

olympics_athletes_events.groupby('oly',as_index=False)['id'].nunique().sort_values(by='id',ascending=False).head(1)

## Top Businesses With Most Reviews `Yelp`

Find the top 5 businesses with most reviews. Assume that each row has a unique business_id such that the total reviews for each business is listed on each row. Output the business name along with the total number of reviews and order your results by the total reviews in descending order.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
yelp_business.sort_values(by='review_count',ascending=False).head(5)[['name','review_count']]

## Reviews of Categories `Yelp`

Calculate number of reviews for every business category. Output the category along with the total number of reviews. Order by total reviews in descending order.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
expnd = yelp_business['categories'].str.split(';',expand=True)
df = pd.concat([yelp_business,expnd],axis=1)

df.melt(id_vars=yelp_business.columns)[['value','review_count']].groupby('value',as_index=False)['review_count'].sum().sort_values(by='review_count',ascending=False)

## Top Cool Votes `Yelp`

Find the review_text that received the highest number of  cool votes.
Output the business name along with the review text with the highest number of cool votes.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = yelp_reviews.groupby('business_name')['cool'].sum().sort_values(ascending=False).reset_index()
df['ranks'] = df['cool'].rank(ascending=False,method='dense')
yelp_reviews[yelp_reviews['business_name'].isin(df.query('ranks == 1')['business_name'].values)][['business_name','review_text']]

## Income By Title and Gender `City of San Francisco`

Find the average total compensation based on employee titles and gender. Total compensation is calculated by adding both the salary and bonus of each employee. However, not every employee receives a bonus so disregard employees without bonuses in your calculation. Employee can receive more than one bonus.
Output the employee title, gender (i.e., sex), along with the average total compensation.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = sf_employee.set_index('id').join(sf_bonus.groupby(['worker_ref_id'])['bonus'].sum(),how='right')

df['com'] = df['salary'] + df['bonus']

df.groupby(['employee_title','sex'])['com'].mean().reset_index()

## Matching Similar Hosts and Guests `Airbnb`

Find matching hosts and guests pairs in a way that they are both of the same gender and nationality.
Output the host id and the guest id of matched pair.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
airbnb_hosts.drop_duplicates().merge(airbnb_guests,left_on=['nationality','gender'],right_on=['nationality','gender'])[['host_id','guest_id']]

## Find the percentage of shipable orders `Google`

Find the percentage of shipable orders.
Consider an order is shipable if the customer's address is known.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = orders.merge(customers,left_on='cust_id',right_on='id')

int((df.dropna().shape[0]/df.shape[0])*100)

## Spam Posts `Meta`

Calculate the percentage of spam posts in all viewed posts by day. A post is considered a spam if a string "spam" is inside keywords of the post. Note that the facebook_posts table stores all posts posted by users. The facebook_post_views table is an action table denoting if a user has viewed a post.

In [ ]:
# Import your libraries
import pandas as pd

facebook_posts['is_spam'] = facebook_posts['post_keywords'].apply(lambda x : 'spam' in x)

spam = facebook_posts.groupby('post_date',as_index=False)['is_spam'].sum()

views = facebook_posts.merge(facebook_post_views,on='post_id').groupby('post_date',as_index=False)['post_id'].nunique()

df = spam.merge(views,on='post_date')

df['spam_share'] = (df['is_spam']/df['post_id'])*100

df[['post_date','spam_share']]

## Apple Product Counts `Apple`

We’re analyzing user data to understand how popular Apple devices are among users who have performed at least one event on the platform. Specifically, we want to measure this popularity across different languages. Count the number of distinct users using Apple devices —limited to "macbook pro", "iphone 5s", and "ipad air" — and compare it to the total number of users per language.


Present the results with the language, the number of Apple users, and the total number of users for each language. Finally, sort the results so that languages with the highest total user count appear first.

In [2]:
# Import your libraries
import pandas as pd

# Start writing code
df = playbook_events.merge(playbook_users,on='user_id')

total_users = df.groupby(['language'],as_index=False)['user_id'].nunique()

apple_users = df[df['device'].isin(['macbook pro','iphone 5s','ipad air'])].groupby(['language'],as_index=False)['user_id'].nunique()

total_users.merge(apple_users,on='language',how='left').sort_values(by='user_id_x',ascending=False).fillna(0)

## No Order Customers `Instacart`

Identify customers who did not place an order between 2019-02-01 and 2019-03-01.


Include:


•    Customers who placed orders only outside this date range.
•    Customers who never placed any orders.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
custs = orders[((orders['order_date'] >= '2019-02-01') & (orders['order_date'] <= '2019-03-01'))]['cust_id'].values

df = customers.merge(orders,left_on='id',right_on='cust_id',how='left')

df[~df['id_x'].isin(custs)]['first_name'].drop_duplicates()

## Number Of Units Per Nationality `Airbnb`

We have data on rental properties and their owners. Write a query that figures out how many different apartments (use unit_id) are owned by people under 30, broken down by their nationality. We want to see which nationality owns the most apartments, so make sure to sort the results accordingly.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = airbnb_units.merge(airbnb_hosts,on='host_id').query('unit_type == "Apartment" & age < 30')

df.groupby('nationality',as_index=False)['unit_id'].nunique()

## Ranking Most Active Guests `Airbnb`

Identify the most engaged guests by ranking them according to their overall messaging activity. The most active guest, meaning the one who has exchanged the most messages with hosts, should have the highest rank. If two or more guests have the same number of messages, they should have the same rank. Importantly, the ranking shouldn't skip any numbers, even if many guests share the same rank. Present your results in a clear format, showing the rank, guest identifier, and total number of messages for each guest, ordered from the most to least active.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = airbnb_contacts.groupby('id_guest')['n_messages'].sum().sort_values(ascending=False).reset_index()

df.insert(0,'rankings',df['n_messages'].rank(ascending=False,method='dense'))

df

## Number of Streets Per Zip Code `City of San Francisco`

Count the number of unique street names for each postal code in the business dataset. Use only the first word of the street name, case insensitive (e.g., "FOLSOM" and "Folsom" are the same). If the structure is reversed (e.g., "Pier 39" and "39 Pier"), count them as the same street. Output the results with postal codes, ordered by the number of streets (descending) and postal code (ascending).

In [ ]:
import pandas as pd

sf_restaurant_health_violations['streets'] = sf_restaurant_health_violations['business_address'].apply(lambda x : x.split()).apply(lambda x : x[0] if not x[0].isdigit() else x[1]).str.lower()

df = sf_restaurant_health_violations.copy()

df = df.groupby('business_postal_code',as_index=False)['streets'].nunique()

df.sort_values(by=['streets','business_postal_code'],ascending=[False,True])

## Meta/Facebook Accounts `Meta`

Calculate the ratio of accounts closed on January 10th, 2020 using the fb_account_status table.

In [2]:
# Import your libraries
import pandas as pd

# Start writing code
(fb_account_status[(fb_account_status['status'] == 'closed') & (fb_account_status['status_date'] == '2020-01-10')].shape[0]) / fb_account_status[(fb_account_status['status_date'] == '2020-01-10')].shape[0]

## Share of Active Users `Meta`

Calculate the percentage of users who are both from the US and have an 'open' status, as indicated in the fb_active_users table.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
(fb_active_users.query('country == "USA" & status == "open"').shape[0]/fb_active_users.shape[0])*100

## Finding Purchases `Amazon`

Identify returning active users by finding users who made a second purchase within 7 days of any previous purchase. Output a list of these user_id.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = amazon_transactions.sort_values(by=['user_id','created_at'])
temp = df[['user_id','created_at']].shift(1).rename(columns={'user_id' : 'shifted_id','created_at':'shifted_created'})

final = pd.concat([df,temp],axis=1)

final = final[final['user_id'] == final['shifted_id']]

final['diff'] = (final['created_at'] - final['shifted_created']).astype(str).str.split(' ').str.get(0).astype(int)

final[final['diff'] < 8]['user_id'].unique()


## Premium Accounts `Meta`

You have a dataset that records daily active users for each premium account. A premium account appears in the data every day as long as it remains premium. However, some premium accounts may be temporarily discounted, meaning they are not actively paying—this is indicated by a final_price of 0.


For each of the first 7 available dates in the dataset, count the number of premium accounts that were actively paying on that day. Then, track how many of those same accounts are still premium and actively paying exactly 7 days later, based solely on their status on that 7th day (i.e., both dates must exist in the dataset). Accounts are only counted if they appear in the data on both dates.


Output three columns:

•   The date of initial calculation.

•   The number of premium accounts that were actively paying on that day.

•   The number of those accounts that remain premium and are still paying after 7 days.

In [ ]:
# Import your libraries
import pandas as pd
import numpy as np
# Start writing code
premium_accounts_by_day = premium_accounts_by_day[premium_accounts_by_day['final_price'] != 0]
df = premium_accounts_by_day[premium_accounts_by_day['entry_date'] < '2022-02-14'].groupby('entry_date').agg(
    {'account_id' : ['count',list]}
    )

df = df.stack(level=0).reset_index().drop(columns='level_1')

temp = premium_accounts_by_day[(premium_accounts_by_day['entry_date'] >= '2022-02-14') & (premium_accounts_by_day['entry_date'] <= '2022-02-20')]

temp = temp.groupby('entry_date',as_index=False).agg(
    {'account_id' : list}
    )

temp['entry_date'] = temp['entry_date'] - pd.to_timedelta(7, unit='d')
df = temp.merge(df,on='entry_date')

df['premium_after7'] = df.apply(lambda x : np.intersect1d(x['account_id'],x['list']),axis=1)

df['num_paid_after7'] = df['premium_after7'].apply(len)

df[['entry_date','count','num_paid_after7']]

## Election Results `Deloitte`

The election is conducted in a city and everyone can vote for one or more candidates, or choose not to vote at all. Each person has 1 vote so if they vote for multiple candidates, their vote gets equally split across these candidates. For example, if a person votes for 2 candidates, these candidates receive an equivalent of 0.5 vote each. Some voters have chosen not to vote, which explains the blank entries in the dataset.


Find out who got the most votes and won the election. Output the name of the candidate or multiple names in case of a tie.
To avoid issues with a floating-point error you can round the number of votes received by a candidate to 3 decimal places.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = voting_results.dropna()

votes = (1/df['voter'].value_counts()).reset_index()

df.merge(votes,on='voter').groupby('candidate')['count'].sum().sort_values(ascending=False).head(1).reset_index()['candidate']

## Flags per Video `Netflix`

For each video, find how many unique users flagged it. A unique user can be identified using the combination of their first name and last name. Do not consider rows in which there is no flag ID.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = user_flags.dropna(subset='flag_id')

df['name'] = df['user_firstname'].fillna('a') + ' ' + df['user_lastname']

df.groupby('video_id',as_index=False)['name'].nunique()

## User with Most Approved Flags `Google`

Which user flagged the most distinct videos that ended up approved by YouTube? Output, in one column, their full name or names in case of a tie. In the user's full name, include a space between the first and the last name.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = flag_review[flag_review['reviewed_outcome'] == 'APPROVED'].merge(user_flags,on='flag_id')

df['name'] = df['user_firstname'] + ' ' + df['user_lastname']

df = df.groupby('name',as_index=False)['video_id'].nunique().sort_values(by='video_id',ascending=False)

df[df['video_id'] == df['video_id'].max()]['name']

## Find Students At Median Writing `General Assembly`

Identify the IDs of students who scored exactly at the median for the SAT writing section.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
sat_scores[sat_scores['sat_writing'] == sat_scores['sat_writing'].median()]['student_id']

## Host Popularity Rental Prices `Airbnb`

You are given a table named airbnb_host_searches that contains data for rental property searches made by users. Determine the minimum, average, and maximum rental prices for each popularity-rating bucket. A popularity-rating bucket should be assigned to every record based on its number_of_reviews (see rules below).


The host’s popularity rating is defined as below:
-   0 reviews: "New"
-   1 to 5 reviews: "Rising"
-   6 to 15 reviews: "Trending Up"
-   16 to 40 reviews: "Popular"
-   More than 40 reviews: "Hot"


Tip: The id column in the table refers to the search ID.


Output host popularity rating and their minimum, average and maximum rental prices. Order the solution by the minimum price.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code

def buckets(x):
    if x > 40:
        return 'Hot'
    elif x > 15:
        return 'Popular'
    elif x > 5:
        return 'Trending Up'
    elif x > 0:
        return 'Rising'
    else:
        return 'New'

airbnb_host_searches['pop'] = airbnb_host_searches['number_of_reviews'].apply(buckets)

df = airbnb_host_searches.groupby('pop').agg(
    {
        'price' : ['min','mean','max']
    }
    ).stack(level=0).reset_index().drop(columns='level_1').sort_values(by='min')

## Top 10 Songs 2010 `Spotify`

Find the top 10 ranked songs in 2010. Output the rank, group name, and song name, but do not show the same song twice. Sort the result based on the rank in ascending order.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = billboard_top_100_year_end.query('year == 2010').groupby('song_name',as_index=False).agg(
    {
        'year_rank' : 'max',
        'group_name' : 'unique'
    }).sort_values(by='year_rank')

df['group_name'] = df['group_name'].apply(lambda x : ''.join(x))

df.head(10)